In [1]:
import numpy as np
import torch
import torchvision
from tqdm import tqdm
from torchmetrics.functional import structural_similarity_index_measure as ssim
from official_api.facepp import face_compare

from AdvFaceGAN import Generator
from utils.dataset import Dataset

print("------------------------加载测试集-------------------------")
test_set = Dataset("C:/yy/datasets/lfw/lfw-aligned-112x112/", "target")
print("-----------------------启动分批队列------------------------")
batch_size = 8
test_set.start_batch_queue(
    batch_size=batch_size,
    batch_format="random_samples",
    transforms=torchvision.transforms.Compose([
        torchvision.transforms.Resize((112,112)),  # 调整图片大小
        torchvision.transforms.ToTensor(),  # 图片转为Tensor [0,1]
        torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # 归一化
    ]),
    num_threads=2,
)
print("-----------------------加载生成器------------------------")
fake_generator = Generator(is_target=True).eval().cuda()
model_generator_dict = torch.load(r"C:\yy\source\Desktop\读研是一条艰苦的道路\1. 论文\做实验\myCode\myCode\4 FrAdv\AdvFaceGAN\save_dir\target 3 8白盒 奇怪ssim\model\00850_generator.pth")
fake_generator.load_state_dict(model_generator_dict)

print("-----------------------开始api测试------------------------")
with torch.no_grad():
    epoch = 750
    after = []
    x=0
    for i in tqdm(range(0, epoch), total=epoch):
        batch = test_set.pop_batch_queue()
        source_faces = batch['sources'].cuda()
        target_faces = batch['targets'].cuda()
        perts, fake_afters = fake_generator.forward(source_faces, target_faces)
        for j in range(0, batch_size):
            torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './test/target.png', nrow=1)
            torchvision.utils.save_image([fake_afters[j] * 0.5 + 0.5], './test/fake.png', nrow=1)
            res = face_compare(face1_path='./test/target.png', face2_path='./test/fake.png')
            if res is None:
                x = x+1
                torchvision.utils.save_image([source_faces[j] * 0.5 + 0.5], './wrong/facepp/source{}_{}.png'.format(x, batch['sources_name'][j]), nrow=1)
                torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './wrong/facepp/target{}_{}.png'.format(x, batch['targets_name'][j]), nrow=1)
                continue
            if res is not None:
                after.append(res)
    
    print(np.mean(after))
    after = np.array(after)
    print(np.sum(after > 62.327) / len(after))
    print(np.sum(after > 69.101) / len(after))
    print(np.sum(after > 73.975) / len(after))
    

'''
lfw ：
target 3 8白盒 无stloss
73.76157870060281
0.8590087073007368
0.7168452779638312
0.5611185532484929

target 4 8白盒 无stloss
77.8469291931704
0.9323736190157349
0.8421493136926682
0.7216270505523937

target 4 8白盒 奇怪ssim 97ssim
74.95894014623363
0.8831831321203877
0.7616051691889134
0.6107804795102874

celeba_hq 
target 4 8白盒 无stloss
77.13664204545454
0.9116161616161617
0.7916666666666666
0.6698232323232324
0.9243806028366088

target 4 8白盒 奇怪ssim
75.88352572145546
0.8795483061480552
0.7641154328732748
0.6398996235884568
0.9430139261484146

target 3 8白盒 无stloss
72.95020372545729
0.8234603121329082
0.6746098338647424
0.527437489511663
0.949701600710551

target 3 8白盒 奇怪ssim
70.33600285762313
0.7599596570852244
0.5878298873760296
0.4390653891410321
0.9642939462661743

'''

------------------------加载测试集-------------------------
13233 images of 5749 classes loaded
-----------------------启动分批队列------------------------
-----------------------加载生成器------------------------


C:\Users\28769\AppData\Local\Temp\ipykernel_16532\548654313.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_generator_dict = torch.load(r"C:\yy\source\Desktop\读研是

-----------------------开始api测试------------------------


  0%|          | 0/750 [00:00<?, ?it/s]C:\yy\installed_software\Anaconda3\envs\AdvFaceGAN\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'


...

error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'


 48%|████▊     | 363/750 [1:13:55<1:20:28, 12.48s/it]

error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'
error: code=401
url=https://api-us.faceplusplus.com/facepp/v3/compare
b'{"error_message":"AUTHENTICATION_ERROR"}\n'


 48%|████▊     | 363/750 [1:14:02<1:18:56, 12.24s/it]


KeyboardInterrupt: 

In [ ]:
import numpy as np
import torch
import torchvision
from tqdm import tqdm
from torchmetrics.functional import structural_similarity_index_measure as ssim
from official_api.facepp import face_compare

from AdvFaceGAN import Generator
from utils.dataset import Dataset

print("------------------------加载测试集-------------------------")
test_set = Dataset("D:/datasets/celeba-hq/celeba_hq-112x112/", "target")
print("-----------------------启动分批队列------------------------")
batch_size = 8
test_set.start_batch_queue(
    batch_size=batch_size,
    batch_format="random_samples",
    transforms=torchvision.transforms.Compose([
        torchvision.transforms.Resize((112,112)),  # 调整图片大小
        torchvision.transforms.ToTensor(),  # 图片转为Tensor [0,1]
        torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # 归一化
    ]),
    num_threads=2,
)
print("-----------------------加载生成器------------------------")
fake_generator = Generator(is_target=True).eval().cuda()
model_generator_dict = torch.load(r"D:\yy\source\Desktop\读研是一条艰苦的道路\1. 论文\做实验\myCode\4 FrAdv\AdvFaceGAN\save_dir\target 3 8白盒 正确ssim 无stloss\model\00650_generator.pth")
fake_generator.load_state_dict(model_generator_dict)

print("-----------------------开始api测试------------------------")
with torch.no_grad():
    epoch = 750
    before = []
    after = []
    ssim_res = []
    x=0
    for i in tqdm(range(0, epoch), total=epoch):
        batch = test_set.pop_batch_queue()
        source_faces = batch['sources'].cuda()
        target_faces = batch['targets'].cuda()
        perts, fake_afters = fake_generator.forward(source_faces, target_faces)
        for j in range(0, batch_size):
            torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './test/target.png', nrow=1)
            torchvision.utils.save_image([fake_afters[j] * 0.5 + 0.5], './test/fake.png', nrow=1)
            res = face_compare(face1_path='./test/target.png', face2_path='./test/fake.png')
            if res is None:
                x = x+1
                torchvision.utils.save_image([source_faces[j] * 0.5 + 0.5], './wrong/facepp/source{}_{}.png'.format(x, batch['sources_name'][j]), nrow=1)
                torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './wrong/facepp/target{}_{}.png'.format(x, batch['targets_name'][j]), nrow=1)
                continue
            if res is not None:
                after.append(res)
        ssim_res.append(ssim(source_faces, fake_afters).item())
    
    print(np.mean(after))
    after = np.array(after)
    print(np.sum(after > 62.327) / len(after))
    print(np.sum(after > 69.101) / len(after))
    print(np.sum(after > 73.975) / len(after))
    
    print(ssim_res)
    print(np.mean(ssim_res))


------------------------加载测试集-------------------------
30000 images of 30000 classes loaded
-----------------------启动分批队列------------------------
-----------------------加载生成器------------------------
-----------------------开始api测试------------------------


  2%|▏         | 16/750 [04:25<3:32:03, 17.33s/it]

error: 'confidence'


  5%|▍         | 34/750 [09:31<3:26:02, 17.27s/it]

error: 'confidence'


...

 73%|███████▎  | 545/750 [2:46:14<1:06:44, 19.53s/it]

error: 'confidence'


 77%|███████▋  | 577/750 [2:56:58<1:00:07, 20.85s/it]

error: 'confidence'


 77%|███████▋  | 578/750 [2:57:18<58:47, 20.51s/it]  

In [ ]:
# # 算TAR
import numpy as np
import torch
import torchvision
from tqdm import tqdm
from official_api.facepp import face_compare

from utils.dataset import Dataset

print("------------------------加载测试集-------------------------")
test_set = Dataset("C:/yy/datasets/celeba-hq/celeba_hq-112x112/", "target")
print("-----------------------启动分批队列------------------------")
batch_size = 8
test_set.start_batch_queue(
    batch_size=batch_size,
    batch_format="random_samples",
    transforms=torchvision.transforms.Compose([
        torchvision.transforms.Resize((112,112)),  # 调整图片大小
        torchvision.transforms.ToTensor(),  # 图片转为Tensor [0,1]
        torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # 归一化
    ]),
    num_threads=2,
    untarget_same=False
)

print("-----------------------开始api测试------------------------")
with torch.no_grad():
    epoch = 200
    before = []
    x=0
    for i in tqdm(range(0, epoch), total=epoch):
        batch = test_set.pop_batch_queue()
        source_faces = batch['sources'].cuda()
        target_faces = batch['targets'].cuda()
        for j in range(0, batch_size):
            torchvision.utils.save_image([source_faces[j] * 0.5 + 0.5], './test/source.png', nrow=1)
            torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './test/target.png', nrow=1)
            res = face_compare(face1_path='./test/target.png', face2_path='./test/source.png')
            if res is None:
                x = x+1
                torchvision.utils.save_image([source_faces[j] * 0.5 + 0.5], './wrong/facepp/source{}_{}.png'.format(x, batch['sources_name'][j]), nrow=1)
                torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './wrong/facepp/target{}_{}.png'.format(x, batch['targets_name'][j]), nrow=1)
            if res is not None:
                before.append(res)
    print(np.mean(before))
    before = np.array(before)
    print(np.sum(after < 62.327) / len(before))
    print(np.sum(after < 69.101) / len(before))
    print(np.sum(after < 73.975) / len(before))

In [ ]:
# 算FAR
import numpy as np
import torch
import torchvision
from tqdm import tqdm
from official_api.aliyun import face_compare

from utils.dataset import Dataset

print("------------------------加载测试集-------------------------")
test_set = Dataset("C:/yy/datasets/celeba-hq/celeba_hq-112x112/", "target")
print("-----------------------启动分批队列------------------------")
batch_size = 8
test_set.start_batch_queue(
    batch_size=batch_size,
    batch_format="random_samples",
    transforms=torchvision.transforms.Compose([
        torchvision.transforms.Resize((112,112)),  # 调整图片大小
        torchvision.transforms.ToTensor(),  # 图片转为Tensor [0,1]
        torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # 归一化
    ]),
    num_threads=2,
    untarget_same=False
)

print("-----------------------开始api测试------------------------")
with torch.no_grad():
    epoch = 200
    before = []
    x=0
    for i in tqdm(range(0, epoch), total=epoch):
        batch = test_set.pop_batch_queue()
        source_faces = batch['sources'].cuda()
        target_faces = batch['targets'].cuda()
        for j in range(0, batch_size):
            torchvision.utils.save_image([source_faces[j] * 0.5 + 0.5], './test/source.png', nrow=1)
            torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './test/target.png', nrow=1)
            res = face_compare(face1_path='./test/target.png', face2_path='./test/source.png')
            if res is None:
                x = x+1
                torchvision.utils.save_image([source_faces[j] * 0.5 + 0.5], './wrong/facepp/source{}_{}.png'.format(x, batch['sources_name'][j]), nrow=1)
                torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './wrong/facepp/target{}_{}.png'.format(x, batch['targets_name'][j]), nrow=1)
            if res is not None:
                before.append(res)
    print(np.mean(before))
    before = np.array(before)
    print(np.sum(after > 62.327) / len(before))
    print(np.sum(after > 69.101) / len(before))
    print(np.sum(after > 73.975) / len(before))